In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool ,cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import ParameterGrid
from itertools import product, chain
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.svm import SVC

ValueError: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 192 from PyObject

In [ ]:
data = pd.read_csv('data.csv')
del data['Unnamed: 0']
data['total_num'] = data['like_num']+data['repost_num']+data['comment_num']
data['at_num'] = data.apply(lambda x:x['content'].count('@'),axis=1)
data['hash_num'] = data.apply(lambda x:x['content'].count('#'),axis=1)
data['has_video'] = data.apply(lambda x:0 if pd.isnull(x['video_url']) else 1,axis=1)
data['has_image'] = data.apply(lambda x:0 if pd.isnull(x['image_url']) else 1,axis=1)
data['is_origin'] = data.apply(lambda x:1 if pd.isnull(x['origin_weibo']) else 0,axis=1)
data['year'] = data.apply(lambda x:pd.to_datetime(x['created_at']).year,axis=1)

def judge_level(x):
    if x['total_num']<=100:
        return 0
    if x['total_num']<=1000:
        return 1
    return 2
data['total_class'] = data.apply(lambda x:judge_level(x),axis=1)

data[['vip_level','fans_num','tweets_num','at_num','hash_num','has_video','has_image','is_origin','year']] = data[['vip_level','fans_num','tweets_num','at_num','hash_num','has_video','has_image','is_origin','year']].apply(pd.to_numeric)


In [ ]:


#dataset = data[['class','vip_level','fans_num','tweets_num','at_num','hash_num','has_video','has_image','is_origin','year']]
#labels = data['total_class']
#_,x_test,_,y_test = train_test_split(dataset,labels,test_size=0.3,random_state=0)

In [ ]:
temp = pd.concat([data[data.total_class == 0].sample(n=2600),data[data.total_class == 1].sample(n=2600),data[data.total_class == 2].sample(n=2600)],axis=0,ignore_index=True)
dataset = temp[['class','vip_level','fans_num','tweets_num','at_num','hash_num','has_video','has_image','is_origin','year']]
labels = temp['total_class']
x_train,_,y_train,_ = train_test_split(dataset,labels,test_size=0.3,random_state=0)

cat_features=['class','vip_level','has_video','has_image','is_origin','year']

cv_dataset =  Pool(data=dataset,
                  label=labels,
                  cat_features=cat_features)

model = CatBoostClassifier(
    iterations=1000 ,cat_features=cat_features, task_type="GPU",devices="0:1:2:3",logging_level = 'silent')

grid = {
    'learning_rate': [0.001,0.005,0.01,0.05, 0.1,0.2],
    'depth':[4, 6,8, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
scoring_evals = {'precision': 'precision_micro', 'recall':'recall_micro' , 'f1': 'f1_micro'}

grid_search =GridSearchCV(model,grid,scoring = scoring_evals)
catboost_result = grid_search.fit(x_train, y_train)

In [ ]:
dataset = pd.get_dummies(temp[['class','vip_level','fans_num','tweets_num','at_num','hash_num','has_video','has_image','is_origin','year']],columns=['class','year','vip_level'])
labels = temp['total_class']
x_train,_,y_train,_ = train_test_split(dataset,labels,test_size=0.3,random_state=0)

rfc = RandomForestClassifier(n_jobs=-1,criterion='gini',max_features= 'auto' ,n_estimators=1500, oob_score = True) 

param_grid = {
    'n_estimators': [900,1000,1100,1200,1300,1400,1500,1600],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion':['gini','entropy'],
}

grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid,scoring = scoring_evals)
randomforest_result = grid_search.fit(x_train,y_train)